## Imports

In [1]:
import torch
import numpy as np
import pickle
import pandas as pd

from sklearn.metrics import matthews_corrcoef, confusion_matrix

from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from torch.nn import CrossEntropyLoss, MSELoss

from tools import *
from multiprocessing import Pool, cpu_count
import convert_examples_to_features

from tqdm import tqdm_notebook, trange
import os
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BertForSequenceClassification
from pytorch_pretrained_bert.optimization import BertAdam, WarmupLinearSchedule

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
logging.basicConfig(level=logging.INFO)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Constants

In [2]:
# The input data dir. Should contain the .tsv files (or other data files) for the task.
DATA_DIR = "../data/"

# Bert pre-trained model selected in the list: bert-base-uncased, 
# bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased,
# bert-base-multilingual-cased, bert-base-chinese.
BERT_MODEL = 'bias_classification.tar.gz'

# The name of the task to train.I'm going to name this 'yelp'.
TASK_NAME = 'bias_classification'

# The output directory where the fine-tuned model and checkpoints will be written.
OUTPUT_DIR = f'../outputs/{TASK_NAME}/'

# The directory where the evaluation reports will be written to.
REPORTS_DIR = f'../reports/{TASK_NAME}_evaluation_reports/'

# This is where BERT will look for pre-trained models to load parameters from.
CACHE_DIR = '../cache/'

# The maximum total input sequence length after WordPiece tokenization.
# Sequences longer than this will be truncated, and sequences shorter than this will be padded.
MAX_SEQ_LENGTH = 128

TRAIN_BATCH_SIZE = 24
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 1
RANDOM_SEED = 42
GRADIENT_ACCUMULATION_STEPS = 1
WARMUP_PROPORTION = 0.1
OUTPUT_MODE = 'classification'

CONFIG_NAME = "config.json"
WEIGHTS_NAME = "pytorch_model.bin"

In [3]:
if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
        REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
        os.makedirs(REPORTS_DIR)
if not os.path.exists(REPORTS_DIR):
    os.makedirs(REPORTS_DIR)
    REPORTS_DIR += f'/report_{len(os.listdir(REPORTS_DIR))}'
    os.makedirs(REPORTS_DIR)

In [4]:
def get_eval_report(task_name, labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    print(len(confusion_matrix(labels, preds).ravel()))
    print(confusion_matrix(labels, preds).ravel())
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    return {
        "task": task_name,
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn
    }

def compute_metrics(task_name, labels, preds):
    assert len(preds) == len(labels)
    return get_eval_report(task_name, labels, preds)

In [5]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained(OUTPUT_DIR + 'vocab.txt', do_lower_case=False)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file ../outputs/bias_classification/vocab.txt


In [6]:
processor = BinaryClassificationProcessor()
eval_examples = processor.get_dev_examples(DATA_DIR)
label_list = processor.get_labels() # [0, 1] for binary classification
num_labels = len(label_list)
eval_examples_len = len(eval_examples)

['2410,2,a,"Therefore , the owners of mortgage-backed securities ( MBS ) and other asset-backed securities ( ABS ) faced not only their declining values but also great uncertainty about what price they might fetch if and when a holder tried to sell them to anyone else ."']
<class 'list'>
1
['2410', '2', 'a', '"Therefore , the owners of mortgage-backed securities ( MBS ) and other asset-backed securities ( ABS ) faced not only their declining values but also great uncertainty about what price they might fetch if and when a holder tried to sell them to anyone else ."']
<class 'list'>
4
['3426,2,a,"Instead of making us more responsible citizens , more inclined to rely on our own good judgment rather than public institutions , justice porn popularizes the idea that the court system is a legitimate venue for mending friendships , punishing moral ( but not criminal ) transgressions , and seeking inspirational hugs from stern but caring authority figures ."']
<class 'list'>
1
['3426', '2', 'a

<class 'list'>
4
['1572,0,a,"During times of class tension and envy , it could introduce a luxury surcharge to create a higher FairTax rate for certain expensive goods such as sports cars or luxury cars ."']
<class 'list'>
1
['1572', '0', 'a', '"During times of class tension and envy , it could introduce a luxury surcharge to create a higher FairTax rate for certain expensive goods such as sports cars or luxury cars ."']
<class 'list'>
4
['3765,1,a,"As Ginsburg suggested , a woman who is determined to have an abortion might suffer more profound anguish at enduring a procedure that her doctor considers less safe , or she might suffer just as much anguish if she ends her pregnancy with another , more common abortion procedure -- one that Kennedy insists is her constitutional right . \'\'"']
<class 'list'>
1
['3765', '1', 'a', '"As Ginsburg suggested , a woman who is determined to have an abortion might suffer more profound anguish at enduring a procedure that her doctor considers less sa

<class 'list'>
4
['1696,0,a,"The act was designed to create a federal effort to address violence against women by providing uniform definitions of abuse and providing funds for victim advocates , women \'s shelters , and counseling ."']
<class 'list'>
1
['1696', '0', 'a', '"The act was designed to create a federal effort to address violence against women by providing uniform definitions of abuse and providing funds for victim advocates , women \'s shelters , and counseling ."']
<class 'list'>
4
['2431,2,a,On the gravestone of its hero -- Ronald Reagan -- is an epithet that suggests how far modern conservatism has traveled from its original basis in theological humility and acknowledgment of original sin : `` I know in my heart that man is good .']
<class 'list'>
1
['2431', '2', 'a', 'On the gravestone of its hero -- Ronald Reagan -- is an epithet that suggests how far modern conservatism has traveled from its original basis in theological humility and acknowledgment of original sin : `

['544,0,a,"But trickle-up economics says that we should cut off the top of the mountain , bring the water down to the village , and watch as it magically defies gravity and somehow trickles back up to the top ."']
<class 'list'>
1
['544', '0', 'a', '"But trickle-up economics says that we should cut off the top of the mountain , bring the water down to the village , and watch as it magically defies gravity and somehow trickles back up to the top ."']
<class 'list'>
4
['3300,2,a,"If we allow the Obama administration to continue to advance its nefarious agenda , America will continue on the path toward becoming an enemy of free speech and religious choice , of individual initiative and the creation of wealth ."']
<class 'list'>
1
['3300', '2', 'a', '"If we allow the Obama administration to continue to advance its nefarious agenda , America will continue on the path toward becoming an enemy of free speech and religious choice , of individual initiative and the creation of wealth ."']
<clas

<class 'list'>
1
['3947', '1', 'a', '"`` Yes , \'\' he said , `` these individuals do have strong philosophic disagreements on various issues , but they all stand for challenging the status quo those special interests who control our federal government ."']
<class 'list'>
4
['1353,0,a,"Whereas earlier advocates had worried about the stringent conditions that were needed for unregulated markets to work their magic , Friedman was the master of clever ( sometimes too clever ) arguments to the effect that those conditions were not really needed , or that they were actually met in real-world markets despite what looked a lot like evidence to the contrary ."']
<class 'list'>
1
['1353', '0', 'a', '"Whereas earlier advocates had worried about the stringent conditions that were needed for unregulated markets to work their magic , Friedman was the master of clever ( sometimes too clever ) arguments to the effect that those conditions were not really needed , or that they were actually met in rea

...

<class 'list'>
4
['3965,1,a,"The efficiency of the system is the product of the optical efficiency ( percent of incident sunlight captured ) times the thermal efficiency ( percent of solar radiation absorbed by the receiving tube ) , times the thermodynamic efficiency of the Rankine cycle generator ."']
<class 'list'>
1
['3965', '1', 'a', '"The efficiency of the system is the product of the optical efficiency ( percent of incident sunlight captured ) times the thermal efficiency ( percent of solar radiation absorbed by the receiving tube ) , times the thermodynamic efficiency of the Rankine cycle generator ."']
<class 'list'>
4
['3879,1,a,"`` In effect , the G20 leaders have activated the IMF \'s power to create money and begin global ` quantitative easing ."']
<class 'list'>
1
['3879', '1', 'a', '"`` In effect , the G20 leaders have activated the IMF \'s power to create money and begin global ` quantitative easing ."']
<class 'list'>
4
['2578,2,a,"His temper flares in public as it did

4
['4003,1,a,"Furthermore , when the Statist exercises authority arbitrarily , substituting his own ideological preferences for the rational decisions of tens of millions of individuals operating in the free market , he not only creates short-term misery , such as shortages and price spikes , but also long-term misery , because he discourages longer-term investment as well ."']
<class 'list'>
1
['4003', '1', 'a', '"Furthermore , when the Statist exercises authority arbitrarily , substituting his own ideological preferences for the rational decisions of tens of millions of individuals operating in the free market , he not only creates short-term misery , such as shortages and price spikes , but also long-term misery , because he discourages longer-term investment as well ."']
<class 'list'>
4
['116,0,a,"Carney points out how the media treat nefarious partnerships between large corporations and government regulators as a surprise or as `` proof that the case for reform is overwhelming , 

['2595,2,a,"To be sure , the assault on speculators is not purely a `` partisan \'\' matter : both Barack Obama and John McCain have blamed these supposed evildoers for raising the costs of energy , although Democratic leaders are more practiced at this scolding ."']
<class 'list'>
1
['2595', '2', 'a', '"To be sure , the assault on speculators is not purely a `` partisan \'\' matter : both Barack Obama and John McCain have blamed these supposed evildoers for raising the costs of energy , although Democratic leaders are more practiced at this scolding ."']
<class 'list'>
4
['3252,2,a,"The responses that followed Banks \' inquiry varied from those who used firearms to save their lives during robbery attempts , to the soundness of mind you have in knowing that you \'re able to defend yourself and your loved ones , to the enjoyment of target shooting and hunting ."']
<class 'list'>
1
['3252', '2', 'a', '"The responses that followed Banks \' inquiry varied from those who used firearms to sa

<class 'list'>
1
['2247', '2', 'a', '"Should the Fed ignore this and simply carry on inflating the money supply , Mises warned , it runs the risk of hyperinflation , a severe , galloping inflation that destroys the currency unit ."']
<class 'list'>
4
['313,0,a,"We are increasingly led to believe that the answers to climate change are primarily to be found in new energy technology , specifically increased energy and carbon efficiencies in both production and consumption ."']
<class 'list'>
1
['313', '0', 'a', '"We are increasingly led to believe that the answers to climate change are primarily to be found in new energy technology , specifically increased energy and carbon efficiencies in both production and consumption ."']
<class 'list'>
4
['2180,2,a,"If a financial institution helps an enterprise borrow other people \'s savings so it can better equip its workers to become more productive sooner , using its own profits , the financial institution has provided a useful service ."']
<cla

['3752,1,a,Gura argues that potential abuse of the Privileges or Immunities Clause should not deter advocates from using it when appropriate .']
<class 'list'>
1
['3752', '1', 'a', 'Gura argues that potential abuse of the Privileges or Immunities Clause should not deter advocates from using it when appropriate .']
<class 'list'>
4
['524,0,a,"The Supreme Court , in a five-to-four decision , found that the preference constituted discrimination and violated the equal protection component of the due process clause of the Constitution .32 The result was correct , and a welcome relief from the discrimination against white males practiced , egregiously , in the name of equality ."']
<class 'list'>
1
['524', '0', 'a', '"The Supreme Court , in a five-to-four decision , found that the preference constituted discrimination and violated the equal protection component of the due process clause of the Constitution .32 The result was correct , and a welcome relief from the discrimination against whit

In [7]:
label_map = {label: i for i, label in enumerate(label_list)}
eval_examples_for_processing = [(example, label_map, MAX_SEQ_LENGTH, tokenizer, OUTPUT_MODE) for example in eval_examples]

In [8]:
process_count = cpu_count() - 1
if __name__ ==  '__main__':
    print(f'Preparing to convert {eval_examples_len} examples..')
    print(f'Spawning {process_count} processes..')
    with Pool(process_count) as p:
        eval_features = list(tqdm_notebook(p.imap(convert_examples_to_features.convert_example_to_feature, eval_examples_for_processing), total=eval_examples_len))

Preparing to convert 4326 examples..
Spawning 7 processes..


In [9]:
all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)

In [10]:
if OUTPUT_MODE == "classification":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
elif OUTPUT_MODE == "regression":
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.float)

In [11]:
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=EVAL_BATCH_SIZE)

In [12]:
# Load pre-trained model (weights)
model = BertForSequenceClassification.from_pretrained(CACHE_DIR + BERT_MODEL, cache_dir=CACHE_DIR, num_labels=len(label_list))

INFO:pytorch_pretrained_bert.modeling:loading archive file ../cache/bias_classification.tar.gz
INFO:pytorch_pretrained_bert.modeling:extracting archive file ../cache/bias_classification.tar.gz to temp dir /tmp/tmp9ip6y7sn
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}



In [13]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, eleme

In [14]:
model.eval()
eval_loss = 0
nb_eval_steps = 0
preds = []

for input_ids, input_mask, segment_ids, label_ids in tqdm_notebook(eval_dataloader, desc="Evaluating"):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)

    with torch.no_grad():
        logits = model(input_ids, segment_ids, input_mask, labels=None)

    # create eval loss and other metric required by the task
    if OUTPUT_MODE == "classification":
        loss_fct = CrossEntropyLoss()
        tmp_eval_loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
    elif OUTPUT_MODE == "regression":
        loss_fct = MSELoss()
        tmp_eval_loss = loss_fct(logits.view(-1), label_ids.view(-1))

    eval_loss += tmp_eval_loss.mean().item()
    nb_eval_steps += 1
    if len(preds) == 0:
        preds.append(logits.detach().cpu().numpy())
    else:
        preds[0] = np.append(
            preds[0], logits.detach().cpu().numpy(), axis=0)

eval_loss = eval_loss / nb_eval_steps
preds = preds[0]
if OUTPUT_MODE == "classification":
    preds = np.argmax(preds, axis=1)
elif OUTPUT_MODE == "regression":
    preds = np.squeeze(preds)
# result = compute_metrics(TASK_NAME, all_label_ids.numpy(), preds)

# result['eval_loss'] = eval_loss

# output_eval_file = os.path.join(REPORTS_DIR, "eval_results.txt")
# with open(output_eval_file, "w") as writer:
#     logger.info("***** Eval results *****")
#     for key in (result.keys()):
#         logger.info("  %s = %s", key, str(result[key]))
#         writer.write("%s = %s\n" % (key, str(result[key])))

In [15]:
test = pd.read_csv('../data/dev.tsv', header=None)
test_labels = test[1].tolist()

In [16]:
y_preds = preds.tolist()

In [17]:
from sklearn.metrics import accuracy_score
print('AUC score : {:.5f}'.format(accuracy_score(test_labels, preds)))

AUC score : 0.19857
